In [1]:
import transformers
print(f"현재 transformers 버전: {transformers.__version__}")

c:\ddddwoo\project\Oneness\AiMind-AiModels\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]

현재 transformers 버전: 4.42.4


In [2]:
import sys
import torch
import types
from unittest.mock import MagicMock
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import os

# ==========================================
# [필살기] Flash Attention 완벽하게 속이기
# ==========================================
# 이미 로드되어 있다면 제거 (충돌 방지)
if "flash_attn" in sys.modules:
    del sys.modules["flash_attn"]

# 1. 진짜 모듈처럼 생긴 껍데기(ModuleType)를 생성
mock_flash_attn = types.ModuleType("flash_attn")

# 2. 파이썬이 의심하지 않도록 명세서(__spec__)와 파일 경로(__file__) 위조
# (아까 난 ValueError는 바로 이 __spec__이 없어서 난 것임)
mock_flash_attn.__spec__ = types.SimpleNamespace(
    name="flash_attn", 
    loader=None, 
    origin="mock", 
    submodule_search_locations=None
)
mock_flash_attn.__file__ = "mock_flash_attn.py"

# 3. Florence-2가 내부에서 호출하는 서브 기능들도 다 가짜로 만듦
mock_flash_attn.bert_padding = MagicMock()
mock_flash_attn.flash_attn_interface = MagicMock()
mock_flash_attn.flash_attn_func = MagicMock()

# 4. 시스템 모듈 목록에 등록
sys.modules["flash_attn"] = mock_flash_attn
sys.modules["flash_attn.bert_padding"] = mock_flash_attn.bert_padding
sys.modules["flash_attn.flash_attn_interface"] = mock_flash_attn.flash_attn_interface

print("✅ Flash Attention 가짜 모듈 등록 완료 (시스템 속이기 성공)")


# ==========================================
# 모델 로드 (이제 진짜 될 거야)
# ==========================================
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = 'microsoft/Florence-2-large'

print(f"{model_id} 로드 시작...")

# 로드 시도
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    trust_remote_code=True, 
    torch_dtype=dtype
).to(device)

processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

print("🎉 드디어 모델 로드 성공! 수고했어 덩우야!")

✅ Flash Attention 가짜 모듈 등록 완료 (시스템 속이기 성공)
microsoft/Florence-2-large 로드 시작...


c:\ddddwoo\project\Oneness\AiMind-AiModels\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jwhon\.cache\huggingface\hub\models--microsoft--Florence-2-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
A new version of the following files was downloaded from https://huggingface.co/microsoft/Flor

🎉 드디어 모델 로드 성공! 수고했어 덩우야!


In [5]:
def run_ocr(image_path):
    """
    이미지 경로를 받아 Florence-2 모델로 텍스트를 추출(OCR)합니다.
    """
    if not os.path.exists(image_path):
        return f"에러: '{image_path}' 파일을 찾을 수 없습니다. 경로를 확인해주세요!"

    # 이미지 열기
    image = Image.open(image_path)
    if image.mode != "RGB":
        image = image.convert("RGB")

    # Florence-2 OCR 태스크 설정
    task_prompt = "<OCR>"

    # 입력 데이터 전처리 (dtype 변수명 수정 완료)
    inputs = processor(text=task_prompt, images=image, return_tensors="pt").to(device, dtype)

    # 생성 (Inference)
    with torch.no_grad():
        generated_ids = model.generate(
            input_ids=inputs["input_ids"],
            pixel_values=inputs["pixel_values"],
            max_new_tokens=1024,
            do_sample=False,
            num_beams=3
        )

    # 결과 디코딩
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text, 
        task=task_prompt, 
        image_size=(image.width, image.height)
    )

    return parsed_answer['<OCR>']

In [8]:
# 파일명이 scanimg1.jpg 맞지?
image_file_path = "scan_result_smart.jpg" 

# OCR 실행
try:
    result_text = run_ocr(image_file_path)
    print("-" * 30)
    print(f"📄 파일명: {image_file_path}")
    print("✨ OCR 결과:")
    print(result_text)
    print("-" * 30)
except Exception as e:
    print(f"❌ 실행 중 에러 발생: {e}")

------------------------------
📄 파일명: scan_result_smart.jpg
✨ OCR 결과:
아
이오이 오자
안이이 마드라
정이마다 (가장이)
있전로리 만만(기림)
고제매막말 (김에)
보린 공적점과맨
박입저절맛
맥
유
강망립
게
그
우감맘맰
여
성
시간
소
원개계갑
수
사
지
예
금
주
국
생
스
관
건
조
부
나
까
릭
동
치
비
모
어
권

------------------------------
